In [24]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

def euclidean_distance(xy1, xy2):
    return np.linalg.norm(np.array(xy1) - np.array(xy2))

locations = {
    "Parul University": (22.3184, 73.1812),
    "Sayaji Baug Zoo": (22.3095, 73.1880),
    "Waghodia Road": (22.3078, 73.1910)
}
destination = (22.3033, 73.1818)  # Vadodara Railway Station


location_names = list(locations.keys())
num_locations = len(location_names)
distance_matrix = np.zeros((num_locations + 1, num_locations + 1))


for i, loc1 in enumerate(location_names):
    for j, loc2 in enumerate(location_names):
        if i != j:
            distance_matrix[i][j] = euclidean_distance(locations[loc1], locations[loc2])
    distance_matrix[i][num_locations] = euclidean_distance(locations[loc1], destination)


for j in range(num_locations):
    distance_matrix[num_locations][j] = euclidean_distance(destination, locations[location_names[j]])


distance_matrix[num_locations][num_locations] = 0

print("Distance Matrix:")
print(distance_matrix)


def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                         data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    if from_node < len(data['distance_matrix']) and to_node < len(data['distance_matrix']):
        return int(data['distance_matrix'][from_node][to_node])
    else:
        raise IndexError("Node index out of range")


distance_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)

solution = routing.SolveWithParameters(search_parameters)


def print_solution(manager, routing, solution):
    index = routing.Start(0)
    route_distance = 0
    route = []
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        if node_index < len(location_names):
            route.append(location_names[node_index])
        next_index = solution.Value(routing.NextVar(index))
        route_distance += distance_matrix[node_index][manager.IndexToNode(next_index)]
        index = next_index
    route.append("Destination")
    route_distance += distance_matrix[manager.IndexToNode(index)][num_locations]

    print(f'Objective: {route_distance} meters')
    print('Route:')
    print(' -> '.join(route))

if solution:
    print_solution(manager, routing, solution)
else:
    print('No solution found!')


Distance Matrix:
[[0.         0.01120045 0.01443607 0.01511192]
 [0.01120045 0.         0.00344819 0.00876812]
 [0.01443607 0.00344819 0.         0.01024158]
 [0.01511192 0.00876812 0.01024158 0.        ]]
Objective: 0.055114047915146826 meters
Route:
Parul University -> Waghodia Road -> Sayaji Baug Zoo -> Destination


In [7]:
# import folium package
import folium


In [8]:
from branca.element import Figure
fig=Figure(width=550,height=350)

In [9]:
coords_1=[22.3184, 73.1812]

coords_2=[22.3095, 73.1880]

coords_3=[22.3078, 73.1910]


In [11]:
 import folium

# Define the coordinates for paths
path_1_coords = [(22.3184, 73.1812), (22.3195, 73.1820), (22.3200, 73.1830)]
path_2_coords = [(22.3184, 73.1812), (22.3200, 73.1850), (22.3210, 73.1860)]

# Define the coordinates for data points
data_points = {
    "Point A": (22.3184, 73.1812),
    "Point B": (22.3195, 73.1820),
    "Point C": (22.3200, 73.1830),
}

# Create a Folium map centered on the initial location
m = folium.Map(location=(22.3184, 73.1812), zoom_start=13)

# Add paths as polylines
folium.PolyLine(locations=path_1_coords, color='blue', weight=5, popup='Path 1').add_to(m)
folium.PolyLine(locations=path_2_coords, color='red', weight=5, popup='Path 2').add_to(m)

# Add markers for data points
for name, coord in data_points.items():
    folium.Marker(location=coord, popup=name, icon=folium.Icon(color='black')).add_to(m)

# Add layer control (optional)
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save("map_with_paths.html")
